In [17]:
import nfl_data_py as nfl
import numpy as np
import pandas as pd

In [18]:
# set year range for data import:
years = list(np.arange(1999,2005))

In [19]:
weekly = nfl.import_weekly_data(years)
nfl.clean_nfl_data(weekly)

In [20]:
len(weekly)

In [21]:
quant_cols = ['completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr',
       'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa',
       'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards',
       'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_air_yards', 'receiving_yards_after_catch',
       'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions',
       'racr', 'target_share', 'air_yards_share', 'wopr', 'special_teams_tds',
       'fantasy_points', 'fantasy_points_ppr']

#per chatgpt: reset_index() is used to restore the default integer-based index and move the current index (which could be a multi-level index) back into regular columns.
grouped_by_week = weekly.groupby(["season", "week", "player_id", "position_group"])[quant_cols].mean().reset_index()

# Group the data by 'position_group' and calculate medians for the selected columns
medians = grouped_by_week.groupby('position_group')[quant_cols].transform('median')

# Replace missing values in the selected columns with corresponding medians
grouped_by_week[quant_cols] = grouped_by_week[quant_cols].fillna(medians)

grouped_by_week

In [22]:
excluded_position_groups = ['DB', 'LB', 'OL', 'SPEC', 'DL']
mask = ~grouped_by_week['position_group'].isin(excluded_position_groups)
grouped_by_week = grouped_by_week[mask]

grouped_by_week

## Optionally Write to postgreSQL server:

In [23]:
# import dependencies

import datetime as dt
from python_to_postgres import python_df_to_postgres
from sqlalchemy import create_engine, URL

In [24]:
# python_df_to_postgres references a local config.ini for user db info
# see example_config.ini
# do not push local .ini to Project github
print(f"Attempting to write the dataframe to postgreSQL: \n")

## dataframe = pandas DataFrame object
## table_name = string value eg: my_table_name'
## if_exists = string value eg:'replace'  # or one of: {‘fail’, ‘replace’, ‘append’, 'replace'}
python_df_to_postgres(grouped_by_week, 'grouped_by_week', 'replace')